In [ ]:
with strategy.scope(): 
    keywordinput = Input(shape=(146), name="keyword")
    keywordRepeat = RepeatVector(250) (keywordinput)
    keywordLstm = Bidirectional(LSTM(units=500, return_sequences=True))(keywordRepeat) #remembers last input

    lstminput = Input(shape=(250, 20), name="sequence")
    lstm1 = Bidirectional(LSTM(units=500, recurrent_dropout=0.3, return_sequences=True))(lstminput) #remembers last input
    lstm2 = Bidirectional(LSTM(units=250, input_dim=20,recurrent_dropout=0.2, return_sequences = True))(lstm1) #remembers last input
    lstm3 = Bidirectional(LSTM(units=100, input_dim=20,recurrent_dropout=0.2, return_sequences = True))(lstm2) #remembers last input
    lstm4 = Bidirectional(LSTM(units=50, input_dim=20,recurrent_dropout=0.2, return_sequences = True))(lstm3) #remembers last input

    addlayer = concatenate([keywordLstm, lstm4])

    output = Dense(8, activation='softmax')(addlayer)
    model = Model(inputs=[keywordinput, lstminput], outputs=output, name="keyword_model")
    model.summary()
    plot_model(model, "keyword_model.png", show_shapes=True)
    
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])


In [ ]:
history = model.fit({"sequence": a_train, "keyword": keywords_train},
                    b_train.astype(np.int8),
                    batch_size=512,
                    validation_split=0.33,
                    epochs=10
                    )

In [ ]:
def dataPrepareKeywords(parsingDict, someDict, keywords):
    keywordsUnmerged = []
    for k, i in parsingDict.items():
        if k[0] in keywords:
            tempBlank = [0 for ent in keywordDict]
            for m in keywords[k[0]]:
                if m in keywordDict:
                    tempBlank[keywordDict[m]] = 1
        else:
            print(k[0])
        #print(k[0])
        #print(tempBlank)
        #print("NEW PROTEIN")
        for j in range(len(i)):
            if i[j] not in someDict:
                i[j] = someDict['?']
            else:
                i[j] = someDict[i[j]]
        keywordsUnmerged.append(tempBlank)
    padded_parsingList = tf.keras.preprocessing.sequence.pad_sequences(
        parsingDict.values(), padding="post", value = someDict['?'], maxlen= 250
    )
    padded_parsingList = padded_parsingList.reshape((padded_parsingList.shape[0], padded_parsingList.shape[1], 1))
    padded_parsingList = np_utils.to_categorical(padded_parsingList)
    print(padded_parsingList.shape)

    return padded_parsingList[:,:, :-1], np.array(keywordsUnmerged)